In this I'm going to test BERTopic for topic modeling. It seems to have significant advantages over LDA.

Going to just follow a tutorial and use their dataset for now

In [ ]:
from bertopic import BERTopic
import pandas as pd
import pickle

In [ ]:
# Since BERT uses sentence embeddings in it's process we want the stop words but the removed ones are useless
# Testing on this one but it's really small so idk if the topics will be good
print(f"Reading in data")
dfCs = []
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments1.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments2.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments3.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments4.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments5.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments6.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments7.json"))

for i in range(len(dfCs)):
    print(f"File {i} shape: {dfCs[i].shape}")



In [ ]:
print(f"Combining data")
dfC = pd.concat(dfCs)
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments2.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments3.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments4.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments5.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments6.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments7.json")])


In [ ]:
print(f"Munging data")

dfC = dfC.loc[(dfC['body'] != '') & (dfC['body'] != '[deleted]') & (dfC['body'] != '[removed]')]
dfC.reset_index(drop=True, inplace=True)


In [37]:
print("sampling data")
dfSample = dfC['body'].sample(1000)
data = dfSample.to_list()

display(dfSample)
len(data)

sampling data


1134883    That we are!! It's always nice to find people ...
773476     I don't use foil. I smoke the tar off of the p...
1658265    Brazil..yes its weaker than Tramadol its 5 tim...
13618      someone who isn't me. It does nothing as far a...
2182275    wow, we get people from literally all over the...
                                 ...                        
3225307    I still feel like this could be an alternate u...
286240     I've never iv'ed opana but I prefer snorted op...
2688987    if you've been on subs for a long period of ti...
1486657    damn whole weeks paycheck? how much do you get...
3422126    where i was living 4 years ago thats whag my d...
Name: body, Length: 1000, dtype: object

1000

In [38]:
with open("data.pkl", "wb") as fOut:
    pickle.dump({'data': data},fOut)


In [39]:
with open("data.pkl", "rb") as fIn:
    cache_data = pickle.load(fIn)
    data = cache_data['data']
    display(len(data))

1000

In [ ]:
# precalculate sentence embeddings so they don't have to be done each time and save them
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
embeddings = embedding_model.encode(data, show_progress_bar=True)

with open("embeddings.pkl", "wb") as fOut:
    pickle.dump({'embeddings-all': embeddings},fOut)



In [ ]:
# Load embeddings from file
with open("embeddings.pkl", "rb") as fIn:
    cache_data = pickle.load(fIn)
    embeddings = cache_data['embeddings']

In [ ]:
# Default representations. Want to try another representation that has less stopwords
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2") 
topics, probs = topicModel.fit_transform(data, embeddings)

In [ ]:
# A maybe better representation?
from bertopic.representation import KeyBERTInspired

topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2", representation_model=KeyBERTInspired()) 
topics, probs = topicModel.fit_transform(data, embeddings)

In [ ]:
topicModel.save("keyBertModelReddit-model", serialization='safetensors')

In [ ]:
topicModel.get_topic_info()

In [ ]:
topicModel.get_topic(2)

In [ ]:
# Want to get this working
topicModel.visualize_distribution(topicModel.probabilities_[0])

In [ ]:
topicModel.visualize_topics()